# **Điểm chuẩn - Crawler w Hocmai**

Crawler dùng để lấy thông tin điểm chuẩn của các trường Đại học qua từng năm.

### **Mục tiêu:**
- Crawl được điểm chuẩn của ~300 trường Đại học tại Việt Nam.
- Crawl được dữ liệu điểm chuẩn của các năm từ 2017-2023 (tối thiểu là 2020-2023).
- Xử lý dữ liệu thô (json) sang dữ liệu dạng bảng (csv, pandas df).
---

### **1. Nhập các thư viện**

In [1]:
# Các thư viện hệ thống, file, hdh
import sys
import os
import time
from time import sleep

# Xử lý đa luồng
from __future__ import print_function
import concurrent.futures

# Các thư viện crawl
import requests
import re
from bs4 import BeautifulSoup
import logging
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Các thư viện xử lý dữ liệu
import pandas as pd
import json

# Các thư viện Selenium cho điều khiển trình duyệt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

#### **2. Lấy danh sách các trường Đại học**

In [2]:
# Hàm gửi yêu cầu GET đến URL và trả về nội dung HTML của trang web
def get_content_request(url='https://huongnghiep.hocmai.vn/diem-chuan/'):
    return requests.get(url).content

# Hàm trích xuất dữ liệu trường đại học từ nội dung HTML
def extract_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    universities_data = []
    
    # Lặp qua từng phần tử (li) chứa thông tin của từng trường đại học trong danh sách
    for e_li in soup.select('#benchmarking > li'):
        e_a = e_li.find('a')
        url = 'https://huongnghiep.hocmai.vn' + e_a.get('href') if e_a.get('href') != '' else None
        university_code = e_a.find('strong').get_text()
        university_name = e_a.get_text().split('-', 1)[-1].strip()
        
        # Tạo đối tượng JSON chứa thông tin của từng trường đại học
        university_obj = {
            'url': url,
            'university_code': university_code,
            'university_name': university_name
        }
        universities_data.append(university_obj)
    
    return universities_data

In [3]:
# Khởi tạo logger
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# Hàm chính, thực hiện lấy và xử lý dữ liệu trường đại học từ trang web
if __name__ == '__main__':
    # URL của trang web cần lấy dữ liệu
    url = 'https://huongnghiep.hocmai.vn/diem-chuan/'
    
    # Gửi yêu cầu GET đến URL và lấy nội dung HTML
    content_html = get_content_request(url=url)
    
    # Trích xuất dữ liệu trường đại học từ nội dung HTML
    universities_data = extract_content(html=content_html)
    logger.info(universities_data)

In [4]:
university_df = pd.DataFrame(universities_data)
university_df 

In [5]:
# Drop các dòng mà có chứa 'cao-dang' trong cột url
university_df = university_df[~university_df['url'].str.contains('cao-dang', case=False, na=False)]
university_df.reset_index(drop=True, inplace=True)
university_df

In [6]:
university_df.to_csv(f"./university.csv",  encoding='utf-8-sig', index=False)

In [7]:
university_df = pd.read_csv(f"./data/university/university.csv")
universities = university_df.to_dict('records')
university_df

,url,university_code,university_name
0,https://huongnghiep.hocmai.vn/diem-truong/dh-t...,TTU,ĐH Tân Tạo
1,https://huongnghiep.hocmai.vn/diem-truong/dai-...,FPT,Đại Học FPT
2,https://huongnghiep.hocmai.vn/diem-truong/dai-...,VU,Đại Học Vinh
3,https://huongnghiep.hocmai.vn/diem-truong/nhac...,HCMCONS,Nhạc Viện TPHCM
4,https://huongnghiep.hocmai.vn/diem-truong/dai-...,HLUV,Đại Học Hoa Lư
...,...,...,...
304,https://huongnghiep.hocmai.vn/diem-truong/phan...,VNUHCM,Phân hiệu Đại học Quốc gia TP.HCM tại Bến Tre
305,https://huongnghiep.hocmai.vn/diem-truong/khoa...,SPAS,Khoa Chính trị - Hành chính (Đại học Quốc gia ...
306,https://huongnghiep.hocmai.vn/diem-truong/dai-...,UMT,Đại học Quản lý và Công nghệ TPHCM
307,https://huongnghiep.hocmai.vn/diem-truong/dai-...,CMC,Đại học CMC


====


In [8]:
# Khởi tạo logger
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# Hàm show_error_info để hiển thị thông tin lỗi
def show_error_info(e):
    logger.error(f"Error: {e}")

# Hàm trích xuất dữ liệu điểm chuẩn sử dụng Selenium
def extract_data_diemchuan_selenium(base_url, university_meta, years):
    # Khởi tạo trình duyệt
    driver = webdriver.Chrome()
    driver.get(base_url)
    time.sleep(5)  # Chờ trang tải

    all_diemchuan_datas = []

    for year in years:
        try:
            # Tìm thẻ <select> với name="year" và chọn năm cụ thể
            year_dropdown = Select(driver.find_element(By.NAME, "year"))
            year_dropdown.select_by_visible_text(str(year))
            time.sleep(5)  # Chờ trang tải lại sau khi chọn năm

            # Phân tích cú pháp HTML của trang web
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Tìm và trích xuất dữ liệu từ bảng
            e_table = soup.select_one('table.hm-table')

            if e_table:
                diemchuan_datas = []
                for e_tr in e_table.select('tbody tr'):
                    e_tds = e_tr.select('td')
                    if len(e_tds) >= 7:
                        major_code = e_tds[3].get_text(strip=True)
                        major_name = e_tds[2].get_text(strip=True)

                        a_tags = e_tds[4].find_all('a')
                        subject_groups = ''.join([a.get_text(strip=True) for a in a_tags])

                        point = e_tds[5].get_text(strip=True)
                        note = e_tds[6].get_text(strip=True)

                        diemchuan_obj = {
                            'university_code': university_meta.get('university_code'),
                            'university_name': university_meta.get('university_name'),
                            'major_code': major_code,
                            'major_name': major_name,
                            'subject_group': subject_groups,
                            'point': point,
                            'note': note,
                            'year': str(year)
                        }
                        logger.info(diemchuan_obj)
                        diemchuan_datas.append(diemchuan_obj)
                
                all_diemchuan_datas.extend(diemchuan_datas)
            else:
                logger.warning(f"Không tìm thấy bảng điểm chuẩn cho năm {year}.")
        
        except Exception as e:
            show_error_info(e)
            continue  # Bỏ qua năm này và tiếp tục với năm tiếp theo
    
    driver.close()
    driver.quit()
    return {'diemchuan_datas': all_diemchuan_datas, 'university_meta': university_meta}

# Hàm thực thi lấy dữ liệu điểm chuẩn cho mỗi đối tượng trường đại học
def method_univerisy_data(university_obj, years):
    university_diemchuan_data = extract_data_diemchuan_selenium(
        base_url=university_obj.get('url'),
        university_meta=university_obj,
        years=years
    )
    return university_diemchuan_data

# Hàm hỗ trợ đa luồng
def multithread_helper(items, method, max_workers=5, timeout_concurrent_by_second=360, debug=False, years=None):
    output = []
    start = time.time()
    print('start:', start)
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_item = {executor.submit(method, item, years): item for item in items}
        #print('loading multithread ' + str(method))

        # Wait for the futures to complete or timeout
        done, not_done = concurrent.futures.wait(future_to_item, timeout=timeout_concurrent_by_second, 
                                                return_when=concurrent.futures.ALL_COMPLETED)

        for future in done:
            item = future_to_item[future]
            try:
                data = future.result()
                if data is not None and data != '':
                    output.append(data)
            except Exception as e:
                print(future)
                print(e)
                print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e).__name__, e)
            else:
                if debug:
                    print('"%s" fetched in %ss' % (item, (time.time() - start)))

    if debug:
        print("Elapsed Time: %ss" % (time.time() - start))
    return output

In [9]:
# # Khởi tạo logger
# logger = logging.getLogger(__name__)
# logging.basicConfig(level=logging.INFO)

# years = [2021]

# # universities = [{'url': 'https://huongnghiep.hocmai.vn/diem-truong/dai-hoc-quy-nhon/', 'university_code': 'QNU', 'university_name': 'Đại Học Quy Nhơn'},
# # {'url': 'https://huongnghiep.hocmai.vn/diem-truong/dai-hoc-noi-vu/', 'university_code': 'HUHA', 'university_name': 'Đại Học Nội Vụ'},
# # {'url': 'https://huongnghiep.hocmai.vn/diem-truong/dai-hoc-dong-a/', 'university_code': 'DAU', 'university_name': 'Đại Học Đông Á'}]

# universities_diemchuan = multithread_helper(items=universities, method=method_univerisy_data,max_workers=10, years=years)

In [10]:
# # Tạo list để chứa các dataframe từ từng đơn vị dữ liệu
# dfs = []

# # Duyệt qua từng đơn vị dữ liệu
# for unit in universities_diemchuan:
#     # Tạo dataframe từ diemchuan_datas
#     df = pd.DataFrame(unit['diemchuan_datas'])
#     # Thêm dataframe vào list
#     dfs.append(df)

# # Ghép các dataframe trong list thành một dataframe lớn
# result_df = pd.concat(dfs, ignore_index=True)

In [11]:
# #result_df = pd.DataFrame(universities_diemchuan)
# result_df.shape

In [12]:
# result_df.to_csv("diemchuan2021_full.csv",  encoding='utf-8-sig', index=False)

2020


In [13]:
# # Khởi tạo logger
# logger = logging.getLogger(__name__)
# logging.basicConfig(level=logging.INFO)

# years2 = [2020]
# universities_diemchuan2 = multithread_helper(items=universities, method=method_univerisy_data,max_workers=10, years=years2)

In [14]:
# # Tạo list để chứa các dataframe từ từng đơn vị dữ liệu
# dfs2 = []

# # Duyệt qua từng đơn vị dữ liệu
# for unit in universities_diemchuan2:
#     # Tạo dataframe từ diemchuan_datas
#     df = pd.DataFrame(unit['diemchuan_datas'])
#     # Thêm dataframe vào list
#     dfs2.append(df)

# # Ghép các dataframe trong list thành một dataframe lớn
# result2_df = pd.concat(dfs2, ignore_index=True)
# result2_df.to_csv("diemchuan2020_full.csv",  encoding='utf-8-sig', index=False)
# result2_df.shape

2019

In [15]:
# # Khởi tạo logger
# logger = logging.getLogger(__name__)
# logging.basicConfig(level=logging.INFO)

# years3 = [2019]
# universities_diemchuan3 = multithread_helper(items=universities, method=method_univerisy_data,max_workers=10, years=years3)

In [16]:
# # Tạo list để chứa các dataframe từ từng đơn vị dữ liệu
# dfs3 = []

# # Duyệt qua từng đơn vị dữ liệu
# for unit in universities_diemchuan3:
#     # Tạo dataframe từ diemchuan_datas
#     df = pd.DataFrame(unit['diemchuan_datas'])
#     # Thêm dataframe vào list
#     dfs3.append(df)

# # Ghép các dataframe trong list thành một dataframe lớn
# result3_df = pd.concat(dfs3, ignore_index=True)
# result3_df.to_csv("diemchuan2019_full.csv",  encoding='utf-8-sig', index=False)
# result3_df.shape

2018

In [17]:
# Khởi tạo logger
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

years4 = [2018]
universities_diemchuan4 = multithread_helper(items=universities, method=method_univerisy_data,max_workers=10, years=years4)

start: 1719229247.589452


INFO:__main__:{'university_code': 'VU', 'university_name': 'Đại Học Vinh', 'major_code': '7510302', 'major_name': 'Công nghệ kỹ thuật điện tử - viễn thông', 'subject_group': 'A00, A01, B00, D01', 'point': '14', 'note': '', 'year': '2018'}
INFO:__main__:{'university_code': 'VU', 'university_name': 'Đại Học Vinh', 'major_code': '7310201A', 'major_name': 'Chính trị học', 'subject_group': 'A00, A01, D01, C00', 'point': '18', 'note': 'chuyên ngành Chính sách công', 'year': '2018'}
INFO:__main__:{'university_code': 'VU', 'university_name': 'Đại Học Vinh', 'major_code': '7520216', 'major_name': 'Kỹ thuật điều khiển và tự động hoá', 'subject_group': 'A00, A01, B00, D01', 'point': '14', 'note': '', 'year': '2018'}
INFO:__main__:{'university_code': 'VU', 'university_name': 'Đại Học Vinh', 'major_code': '7580205', 'major_name': 'Kỹ thuật xây dựng', 'subject_group': 'A00, B00, A01, D01', 'point': '14', 'note': 'Kỹ thuật xây dựng công trình giao thông', 'year': '2018'}
INFO:__main__:{'university_co

<Future at 0x1ec3e435e10 state=finished raised TimeoutException>
Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=126.0.6478.63)
Stacktrace:
	GetHandleVerifier [0x00007FF6B9F23E32+31618]
	(No symbol) [0x00007FF6B9E9B099]
	(No symbol) [0x00007FF6B9D5888A]
	(No symbol) [0x00007FF6B9D4112D]
	(No symbol) [0x00007FF6B9D40E30]
	(No symbol) [0x00007FF6B9D3EDBD]
	(No symbol) [0x00007FF6B9D3F5CF]
	(No symbol) [0x00007FF6B9D4E3BE]
	(No symbol) [0x00007FF6B9D668AF]
	(No symbol) [0x00007FF6B9D6C2DA]
	(No symbol) [0x00007FF6B9D3FD5A]
	(No symbol) [0x00007FF6B9D666CD]
	(No symbol) [0x00007FF6B9DEC95B]
	(No symbol) [0x00007FF6B9DCCEB3]
	(No symbol) [0x00007FF6B9D9A46B]
	(No symbol) [0x00007FF6B9D9B001]
	GetHandleVerifier [0x00007FF6BA229FFD+3202381]
	GetHandleVerifier [0x00007FF6BA276A1D+3516269]
	GetHandleVerifier [0x00007FF6BA26C490+3473888]
	GetHandleVerifier [0x00007FF6B9FD5D36+760454]
	(No symbol) [0x00007FF6B9EA6B3F]
	(No symbol) [0x00007FF6B9EA1CD4]


INFO:__main__:{'university_code': 'MKU', 'university_name': 'Đại Học Cửu Long', 'major_code': '7510102', 'major_name': 'Công nghệ kỹ thuật công trình xây dựng', 'subject_group': 'A00, A01, D01, C01', 'point': '13', 'note': '', 'year': '2018'}
INFO:__main__:{'university_code': 'MKU', 'university_name': 'Đại Học Cửu Long', 'major_code': '7810103', 'major_name': 'Quản trị dịch vụ du lịch và lữ hành', 'subject_group': 'A00, A01, D01, C00', 'point': '13', 'note': '', 'year': '2018'}
INFO:__main__:{'university_code': 'MKU', 'university_name': 'Đại Học Cửu Long', 'major_code': '7580205', 'major_name': 'Kỹ thuật xây dựng', 'subject_group': 'A00, A01, D01, C01', 'point': '13', 'note': 'Kỹ thuật xây dựng công trình giao thông', 'year': '2018'}
INFO:__main__:{'university_code': 'MKU', 'university_name': 'Đại Học Cửu Long', 'major_code': '7510301', 'major_name': 'Công nghệ kỹ thuật điện, điện tử', 'subject_group': 'A00, A01, D01, C01', 'point': '13', 'note': '', 'year': '2018'}
INFO:__main__:{'uni

In [18]:
# Tạo list để chứa các dataframe từ từng đơn vị dữ liệu
dfs4 = []

# Duyệt qua từng đơn vị dữ liệu
for unit in universities_diemchuan4:
    # Tạo dataframe từ diemchuan_datas
    df = pd.DataFrame(unit['diemchuan_datas'])
    # Thêm dataframe vào list
    dfs4.append(df)

# Ghép các dataframe trong list thành một dataframe lớn
result4_df = pd.concat(dfs4, ignore_index=True)
result4_df.to_csv("diemchuan2018_full.csv",  encoding='utf-8-sig', index=False)
result4_df.shape

(327, 8)

---
